In [2]:
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import SpectralClustering
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
from sklearn.datasets import make_biclusters
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from copy import deepcopy
sns.set(color_codes=True)

In [3]:
user_cluster = pd.DataFrame(pd.read_csv('./user_115_features_clusters.csv',header=0))
use_mer_freq_dic = pickle.load(open('./user_merchant_freq.pkl', 'rb'))
new_old_id_maps = pickle.load(open('./new_old_id_maps.pkl', 'rb'))

In [4]:
new_old_id_maps.keys()

dict_keys(['user_new_old_id_maps', 'user_old_new_id_maps', 'sel_new_old_id_maps', 'sel_old_new_id_maps', 'mer_new_old_id_maps', 'mer_old_new_id_maps', 'user_intersection_old_new_id', 'user_intersection_new_old_id'])

In [5]:
training_data_map = pickle.load(open('./training_data_map.pkl', 'rb'))
test_data_map = pickle.load(open('./test_data_map.pkl', 'rb'))

In [6]:
user_cluster[:4]


,new_use_id,old_use_id,taobao_record_count,taobao_buy_count,taobao_click_count,taobao_buy_ratio,taobao_items_buy_count,taobao_items_click_count,taobao_items_buy_ratio,taobao_cats_buy_count,...,svd_feature_92,svd_feature_93,svd_feature_94,svd_feature_95,svd_feature_96,svd_feature_97,svd_feature_98,svd_feature_99,svd_feature_100,cluster_label
0,0,1980536,1948,4,1944,0.002053,4,323,0.012232,2,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.113512,0.000000,0.0,7
1,1,647747,337,8,329,0.023739,8,266,0.029197,5,...,0.0,0.000000,0.0,0.000000,0.000610,0.0,0.060873,0.000196,0.0,8
2,2,183647,431,79,352,0.183295,56,220,0.202899,18,...,0.0,0.000081,0.0,0.000713,0.000251,0.0,0.000013,0.000020,0.0,1
3,3,723956,384,59,325,0.153646,42,203,0.171429,17,...,0.0,0.000204,0.0,0.000000,0.000000,0.0,0.000000,0.030745,0.0,1


In [7]:
use_mer_loc_freq_dic = pickle.load(open('./user_merchant_freq_new_id.pkl', 'rb'))

In [8]:
len(use_mer_loc_freq_dic)

183566

In [9]:
id_list = list(use_mer_loc_freq_dic.keys())

In [10]:
len(id_list)

183566

In [11]:
new_old_id_maps['user_new_old_id_maps']

{0: 1980536,
 1: 647747,
 2: 183647,
 3: 723956,
 4: 1781938,
 5: 1652831,
 6: 416403,
 7: 43130,
 8: 502518,
 9: 109400,
 10: 488933,
 11: 1023483,
 12: 786586,
 13: 1554927,
 14: 958173,
 15: 208174,
 16: 1004531,
 17: 1651174,
 18: 194706,
 19: 445860,
 20: 1158774,
 21: 1622079,
 22: 1034668,
 23: 805086,
 24: 1488584,
 25: 1073773,
 26: 250900,
 27: 1612038,
 28: 988047,
 29: 223687,
 30: 1918824,
 31: 1032167,
 32: 951329,
 33: 856185,
 34: 482155,
 35: 828662,
 36: 1992620,
 37: 466466,
 38: 1996918,
 39: 753366,
 40: 1027731,
 41: 505107,
 42: 326722,
 43: 1712783,
 44: 929801,
 45: 984004,
 46: 2021274,
 47: 731178,
 48: 1101668,
 49: 1790436,
 50: 1684716,
 51: 1891681,
 52: 1928646,
 53: 1558451,
 54: 1496357,
 55: 1900578,
 56: 2019999,
 57: 2012873,
 58: 1921597,
 59: 1574549,
 60: 1877190,
 61: 1284385,
 62: 400276,
 63: 287016,
 64: 471026,
 65: 1978207,
 66: 621310,
 67: 1492829,
 68: 320925,
 69: 917743,
 70: 815220,
 71: 596415,
 72: 1284902,
 73: 1336005,
 74: 130317

In [12]:
for i in range(len(id_list)):
    id_list[i] = new_old_id_maps['user_new_old_id_maps'][id_list[i]]

In [13]:
test_data_map

{881482: {172: [4822]},
 695544: {172: [6540, 9668, 820]},
 1974194: {172: [6540]},
 802596: {172: [6540]},
 983759: {172: [6540]},
 1286037: {172: [6540]},
 414796: {172: [6540]},
 278762: {172: [8335, 820]},
 451220: {172: [8335, 820, 820, 820, 820]},
 1293955: {172: [8335, 8335, 820]},
 397789: {172: [8335]},
 1358801: {172: [8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335]},
 934415: {172: [8335, 8335, 8335, 2606, 2606, 2606]},
 588649: {172: [8335, 8335, 820, 820, 820, 2606, 2606, 2606, 2606]},
 1641310: {172: [8335]},
 1231252: {172: [8335]},
 903901: {172: [8335]},
 199775: {172: [8335, 2606]},
 235331: {172: [8335, 2606, 2606, 2606, 2606, 2606]},
 152533: {172: [8335]},
 1729064: {172: [8335]},
 1268147: {172: [8335, 8335, 8335, 820, 2606, 9793]},
 538540: {172: [8335]},
 360180: {172: [8335, 8335]},
 282796: {172: [8335, 8335]},
 451795: {172: [8335]},
 809696: {172: [8335, 820, 820]},
 274026: {172: [8335]},
 78

In [14]:
# filtering test data
new_test_data_map = {}
for i in id_list:
    if i in test_data_map:
        new_test_data_map[i] = test_data_map[i]
print(len(new_test_data_map))

63795


In [15]:
new_test_data_map

{1027765: {172: [820]},
 859759: {172: [4822]},
 1975225: {172: [820, 575]},
 357648: {172: [820, 2606]},
 1115889: {172: [820, 820, 820, 820, 820, 2606]},
 408834: {172: [263, 263, 263]},
 695544: {172: [6540, 9668, 820]},
 1974194: {172: [6540]},
 1925368: {172: [820]},
 802596: {172: [6540]},
 983759: {172: [6540]},
 86873: {172: [820]},
 781631: {172: [2606, 2606, 2606]},
 1286037: {172: [6540]},
 414796: {172: [6540]},
 238319: {172: [263]},
 451220: {172: [8335, 820, 820, 820, 820]},
 1293955: {172: [8335, 8335, 820]},
 1688048: {172: [820, 820, 820, 820, 820, 2606, 2606, 2606]},
 1056558: {172: [820, 820, 820, 820, 820, 820]},
 397789: {172: [8335]},
 1358801: {172: [8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335,
   8335]},
 934415: {172: [8335, 8335, 8335, 2606, 2606, 2606]},
 747984: {172: [820, 820]},
 588649: {172: [8335, 8335, 820, 820, 820, 2606, 2606, 2606, 2606]},
 1231252: {172: [8335]},
 199775: {172: [8335, 260

In [16]:
# using old id system
# depend on user_cluster
from sklearn.metrics.pairwise import cosine_similarity
import heapq
def find_similar_user(user_id, top_k, user_df):
    user_feature = user_df[user_df.old_use_id == user_id]
    cluster = int(user_feature['cluster_label'])
#     print( cluster)
    same_cluster_df = user_df[user_df.cluster_label == cluster]
    user_id_list = list(same_cluster_df['old_use_id'])
    k_result = []
    curr_min = 0.0
    user_feature_array = np.array(user_feature.iloc[:, 2:115])
    # n * 1 matrix
    cosine_matrix = cosine_similarity(np.array(same_cluster_df.iloc[:, 2:115]), user_feature_array)
    cosine_matrix = cosine_matrix.reshape(len(cosine_matrix), )
    cosine_list = list(cosine_matrix)
    cosine_list = [-cosine_list[i] for i in range(len(cosine_list))]
#     print(cosine_list[:10])
    new_list = list(zip(cosine_list, user_id_list))
#     print(new_list[:10])
    heapq.heapify(new_list)
#     q = heapq.heappop(new_list)
#     print(q)
    for i in range(top_k):
        q = heapq.heappop(new_list)
        k_result.append(q[1])
    return k_result
        

In [17]:
print(find_similar_user(1683062, 2, user_cluster))

[1683062, 780377]


In [18]:
len(training_data_map)

230496

In [19]:
# test data map
# precision and recall and F1 and the same format recommendation list
def User_CF(new_test_data_map, top_k, list_len, sample_data):
    recommend_list = {}
    precision_list = []
    recall_list = []
    count = 1
    for user in tqdm(new_test_data_map):
        if count > sample_data:
                break
        for loc in new_test_data_map[user]:
            sim_users = find_similar_user(user, top_k, user_cluster)
            recommend_list[(user, loc)] = []
#             print(sim_users)
            for i in sim_users:
                if i in training_data_map:
                    for j in training_data_map[i]:
                        recommend_list[(user, loc)].extend(training_data_map[i][j])
#             print(recommend_list[(user, loc)])
            # candidate
            recommend_list[(user, loc)] = sorted(recommend_list[(user, loc)], key=lambda x: x[1], reverse=True)[:list_len]
            tmp_test_set = set(new_test_data_map[user][loc])
            tmp_recommend_list = set([recommend_list[(user, loc)][i][0] for i in range(len(recommend_list[(user, loc)]))])
#             print(tmp_test_set, tmp_recommend_list)
            m,n = len(tmp_test_set), len(tmp_recommend_list)
            if m<=0 or n <= 0:
                continue
#             tmp = [val for val in new_test_data_map[user][loc] if val in tmp_recommend_list]
            tmp_res = len(tmp_test_set.intersection(tmp_recommend_list))
#             print(tmp_res)
            precision_list.append(float(tmp_res/n))
            recall_list.append(float(tmp_res/m))
            count += 1
    ap = sum(precision_list)/len(precision_list)
    ar = sum(recall_list)/len(recall_list)
    f1 = 0.5*ap*ar/(ap+ar)
    print(ap, ar, f1)
    return recommend_list
                

In [20]:
#5, 5: 0.3299999999999998 0.8450000000000002 0.11865957446808505
#5, 5: 0.33233333333333365 0.9216666666666666 0.4885176324650013
User_CF(new_test_data_map, 5, 5, 300)

  0%|          | 302/63795 [02:24<8:27:26,  2.09it/s]

0.33233333333333365 0.9216666666666666 0.12212940811625032


{(2004, 172): [(5231, 5), (7755, 5), (1817, 4), (7155, 3), (493, 2)],
 (2855, 172): [(414, 15), (7144, 12), (5530, 9), (5530, 9), (8335, 8)],
 (3663, 172): [(8248, 7), (820, 6), (5231, 3), (5231, 2), (9485, 2)],
 (3717, 172): [(83, 5), (820, 2)],
 (10343, 172): [(820, 5), (6880, 4), (2183, 3), (630, 3), (2075, 3)],
 (12779, 172): [(820, 8), (5530, 7), (2606, 7), (6691, 6), (820, 6)],
 (23121, 172): [(820, 5), (9968, 5), (820, 5), (1556, 4), (5231, 4)],
 (24874, 172): [(3498, 12), (9673, 7), (820, 6), (820, 6), (5231, 5)],
 (30888, 172): [(820, 9), (2606, 7), (820, 5), (4342, 5), (820, 4)],
 (37724, 172): [(6114, 5), (4264, 5), (424, 5), (2606, 5), (5231, 4)],
 (56910, 172): [(5208, 14), (515, 9), (820, 8), (5244, 6), (9580, 5)],
 (57121, 172): [(820, 10), (820, 9), (5231, 8), (764, 7), (4968, 7)],
 (67340, 172): [(820, 16), (820, 10), (820, 10), (820, 10), (115, 8)],
 (68203, 172): [(820, 7), (2641, 6), (3261, 6), (128, 6), (5231, 6)],
 (69211, 172): [(820, 12), (820, 8), (4264, 7), (8

In [21]:
0.12212940811625032*4

0.4885176324650013